In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

In [3]:
a = pd.read_csv("train.csv")
a.drop("Id",axis=1,inplace=True)
a.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [5]:
a.isna().sum()[a.isna().sum()>0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [6]:
obj=a.select_dtypes(include=['object']).columns
obj

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [7]:
for col in obj:
  a[col]=a[col].fillna(a[col].mode()[0])

In [8]:
integer=a.select_dtypes(include=['float64','int64']).columns
integer

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [9]:
for col in integer:
  a[col]=a[col].fillna(a[col].mean())

In [10]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
for col in obj:
  a[col]=lb.fit_transform(a[col])

In [11]:
a.corr()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
MSSubClass,1.000000,0.035900,-0.357056,-0.139781,-0.024969,0.184539,0.119289,-0.002940,-2.284384e-02,0.075910,...,0.008283,-0.001688,0.026310,-0.019705,-0.007683,-0.013585,-0.021407,0.012464,-0.024940,-0.084284
MSZoning,0.035900,1.000000,-0.106363,-0.034452,0.087654,-0.329278,0.061887,-0.017854,-1.192034e-03,-0.009895,...,-0.003128,0.002264,-0.027981,-0.010701,0.009293,-0.031496,-0.020628,0.097437,0.009494,-0.166872
LotFrontage,-0.357056,-0.106363,1.000000,0.306795,-0.037323,-0.171727,-0.144931,-0.075647,-5.447825e-18,-0.181253,...,0.180868,-0.152692,-0.035853,0.007192,0.001168,0.010158,0.006768,-0.030846,0.058464,0.334901
LotArea,-0.139781,-0.034452,0.306795,1.000000,-0.197131,-0.077747,-0.165315,-0.149083,1.012318e-02,-0.121161,...,0.077672,-0.038048,0.005196,-0.009333,0.038068,0.001205,-0.014261,0.012292,0.034169,0.263843
Street,-0.024969,0.087654,-0.037323,-0.197131,1.000000,0.010919,-0.010224,0.115995,1.681767e-03,0.013960,...,0.004413,-0.003194,-0.016083,-0.002536,-0.022733,0.003690,-0.025043,0.014339,0.006064,0.041036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MoSold,-0.013585,-0.031496,0.010158,0.001205,0.003690,-0.021780,-0.033455,-0.011599,-5.155166e-02,0.018902,...,-0.033737,0.030449,-0.019086,-0.004063,-0.006495,1.000000,-0.145721,-0.047386,0.013320,0.046432
YrSold,-0.021407,-0.020628,0.006768,-0.014261,-0.025043,-0.001392,0.036449,0.020507,2.335254e-02,-0.005992,...,-0.059689,0.055514,-0.014833,-0.011428,0.004906,-0.145721,1.000000,-0.002327,0.003880,-0.028923
SaleType,0.012464,0.097437,-0.030846,0.012292,0.014339,-0.005434,-0.000911,-0.025754,-1.267697e-01,0.014325,...,0.011009,-0.015605,-0.007421,-0.012393,0.015773,-0.047386,-0.002327,1.000000,0.184067,-0.054911
SaleCondition,-0.024940,0.009494,0.058464,0.034169,0.006064,-0.028603,-0.038118,0.033809,-8.970068e-02,0.051579,...,-0.067251,0.110109,0.048126,-0.008233,0.013027,0.013320,0.003880,0.184067,1.000000,0.213092


In [13]:
x=a.drop(['SalePrice'],axis=1)
y=a['SalePrice']

In [15]:
b = "test.csv"
test = pd.read_csv(b)
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [16]:
test.isna().sum()[test.isna().sum()>0]

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64

In [17]:
obj1=test.select_dtypes(include=['object']).columns
obj1

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [18]:
for col in obj1:
  test[col]=test[col].fillna(test[col].mode()[0])

In [19]:
integer1=test.select_dtypes(include=['float64','int64']).columns
integer1

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [20]:
for col in integer1:
  test[col]=test[col].fillna(test[col].mean())

In [21]:
lb1=LabelEncoder()
for col in obj:
  test[col]=lb1.fit_transform(test[col])

In [22]:
learning_rate=[0.01,0.1,1,0.2,0.4]

In [23]:
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.metrics import r2_score

Gradient Boosting Regressor

In [24]:
for r in learning_rate:
  gbc=GradientBoostingRegressor(n_estimators=100,learning_rate=r,random_state=42)
  gbc.fit(x,y)
  train_predict=gbc.predict(x)
  print("For learning rate :",r)
  print("Accuracy of training :",r2_score(y,train_predict))
  print()

For learning rate : 0.01
Accuracy of training : 0.7020668284749326

For learning rate : 0.1
Accuracy of training : 0.9637089251345802

For learning rate : 1
Accuracy of training : 0.9947526516173985

For learning rate : 0.2
Accuracy of training : 0.9778950264264408

For learning rate : 0.4
Accuracy of training : 0.9878513533515493



In [25]:
r=1
gbc=GradientBoostingRegressor(learning_rate=r,random_state=42)
gbc.fit(x,y)
train_predict=gbc.predict(x)
print("For learning rate :",r)
print("Accuracy of training :",r2_score(y,train_predict))
print()

For learning rate : 1
Accuracy of training : 0.9947526516173985



In [26]:
test_id = test.Id
test = test.drop('Id',axis=1)

Ada Boost Regressor

In [27]:
for r in learning_rate:
  adb=AdaBoostRegressor(n_estimators=100,learning_rate=r,random_state=42)
  adb.fit(x,y)
  train_predict=gbc.predict(x)
  print("For learning rate :",r)
  print("Accuracy of training :",r2_score(y,train_predict))
  print()

For learning rate : 0.01
Accuracy of training : 0.9947526516173985

For learning rate : 0.1
Accuracy of training : 0.9947526516173985

For learning rate : 1
Accuracy of training : 0.9947526516173985

For learning rate : 0.2
Accuracy of training : 0.9947526516173985

For learning rate : 0.4
Accuracy of training : 0.9947526516173985



In [28]:
adb=AdaBoostRegressor(n_estimators=100,learning_rate=r,random_state=42)
adb.fit(x,y)
train_predict=gbc.predict(x)
print("For learning rate :",r)
print("Accuracy of training :",r2_score(y,train_predict))
print()

For learning rate : 0.4
Accuracy of training : 0.9947526516173985



XGBoost Regressor

In [29]:
from xgboost import XGBRegressor
xgb=XGBRegressor(n_estimators=100)
xgb.fit(x,y)
train_predict=xgb.predict(x)
print("For learning rate :",r)
print("Accuracy of training :",r2_score(y,train_predict))
print()

For learning rate : 0.4
Accuracy of training : 0.9994952644430615



In [30]:
y_pred = xgb.predict(test)

In [31]:
output = pd.DataFrame({'Id':test_id,'SalePrice':y_pred})
output.to_csv('prediction2.csv',index=False)